In [1]:
from utils.lyric_collect import get_whole_song_lyrics
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import re
from urllib.parse import urljoin
import neologdn
import pickle
from datetime import datetime as dt


In [ ]:
from sudachipy import dictionary, tokenizer as sudachi_tokenizer
import re
from emoji import replace_emoji
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
EMBED_MODEL_NAME ="paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

In [ ]:
docs = fashion["cleaned_content"].fillna("").tolist()

# VectorizerにSudachiトークナイザを渡す
vectorizer = CountVectorizer(tokenizer=sudachi_tokenize, lowercase=False)

# BERTopic本体
topic_model = BERTopic(
    embedding_model=embed_model,
    vectorizer_model=vectorizer,
    # 雑多集合を1つにまとめたいなら ↓
    calculate_probabilities=False,
    min_topic_size=30,      # 中規模クラスタを出したいなら 20〜80 で調整
    nr_topics=None,         # 自動
    top_n_words=10
)

topics, probs = topic_model.fit_transform(docs)

In [ ]:
sudachi_dict = dictionary.Dictionary(dict="full").create()
SPLIT_MODE = sudachi_tokenizer.Tokenizer.SplitMode.C

# 軽いストップワード（必要に応じて足してOK）
STOPWORDS = set("""
する なる ある いる こと これ それ あれ の に は が を と て で も です ます でした でした
よう ため もの ない ん から まで そして また など ので とか より
公式 広告 PR キャンペーン お知らせ
""".split())

In [2]:
ROOT = Path("./").resolve().parent
SECRETS = ROOT / ".secrets"
DATA = ROOT / "data"
DATA.mkdir(exist_ok=True,parents=True)

In [3]:
res = requests.get("https://www.uta-net.com/ranking/artist/")
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
ranking = pd.DataFrame()
container = soup.select_one("#rank-total-tab")
for i,li in enumerate(container.select("ol.ranking-ol-table li")):
    rank = li.find("div", class_="text-center").get_text(strip=True)
    name = li.find_all("div")[1].get_text(strip=True)
    href = li.find_parent("a")["href"]  # アーティストページURL
    id = href.rstrip("/").split("/")[-1]
    ranking.loc[i,"rank"] = rank
    ranking.loc[i,"artist_id"] = id
    ranking.loc[i,"name"] = name

In [5]:
LYRICS = DATA / "lyrics"
LYRICS.mkdir(exist_ok=True,parents=True)
execution_date = dt.today().strftime("%Y%m%d")
sample_n = 50
mode = 4

for row in ranking[["artist_id","name"]].iterrows():
    artist_id = row[1]["artist_id"]
    artist_name = row[1]["name"]
    if artist_id == "4002":
        continue
    else:
        data = get_whole_song_lyrics(artist_id, "artist", mode, sample_n,1)
        with open(LYRICS / f"{artist_name}_{execution_date}_mode{mode}_top{sample_n}.pkl", mode='wb') as f:
            pickle.dump(data, f)

back numberの歌詞一覧リスト125曲を取得します
全125曲完了                                                                                                             
嵐の歌詞一覧リスト361曲を取得します
全400曲完了                                                                                                             
RADWIMPSの歌詞一覧リスト186曲を取得します
全186曲完了                                                                                                             
Mr.Childrenの歌詞一覧リスト260曲を取得します
全400曲完了                                                                                                             
米津玄師の歌詞一覧リスト114曲を取得します
全114曲完了                                                                                                             
西野カナの歌詞一覧リスト180曲を取得します
全180曲完了                                                                                                             
GReeeeNの歌詞一覧リスト182曲を取得します
全182曲完了                                                                                                             
BU

In [8]:
def pickle_load(path:str):
    with open(path, mode='rb') as f:
        return pickle.load(f)

In [ ]:
list(LYRICS)

In [11]:
data = [pickle_load(path) for path in LYRICS.glob("*.pkl")]

In [14]:
data[0]

[{'title': 'クリスマスソング',
  'artist': 'back number',
  'lyricist': '清水依与吏',
  'composer': '清水依与吏',
  'arranger': 'back number・小林武史・四家卯大',
  'lyrics_url': 'https://www.uta-net.com/song/195484/',
  'lyric': 'どこかで鐘が鳴って らしくない言葉が浮かんで 寒さが心地よくて あれ なんで恋なんかしてんだろう 聖夜だなんだと繰り返す歌と わざとらしくきらめく街のせいかな 会いたいと思う回数が 会えないと痛いこの胸が 君の事どう思うか教えようとしてる いいよ そんな事自分で分かってるよ サンタとやらに頼んでも仕方ないよなぁ できれば横にいて欲しくて どこにも行って欲しくなくて 僕の事だけをずっと考えていて欲しい でもこんな事を伝えたら格好悪いし 長くなるだけだからまとめるよ 君が好きだ はしゃぐ恋人達は トナカイのツノなんか生やして よく人前で出来るなぁ いや 羨ましくなんてないけど 君が喜ぶプレゼントってなんだろう 僕だけがあげられるものってなんだろう 大好きだと言った返事が 思ってたのとは違っても それだけで嫌いになんてなれやしないから 星に願いをなんてさ 柄じゃないけど 結局君じゃないと嫌なんだって 見上げてるんだ あの時君に 出会って ただそれだけで 自分も知らなかった自分が次から次に 会いたいと毎日思ってて それを君に知って欲しくて すれ違う人混みに君を探している こんな日は他の誰かと笑ってるかな 胸の奥の奥が苦しくなる できれば横にいて欲しくて どこにも行って欲しくなくて 僕の事だけをずっと考えていて欲しい やっぱりこんな事伝えたら格好悪いし 長くなるだけだからまとめるよ 君が好きだ 聞こえるまで何度だって言うよ 君が好きだ',
  'release': '2015/11/18'},
 {'title': '花束',
  'artist': 'back number',
  'lyricist': '清水依与吏',
  'composer': '清水依与吏',
  'arranger': '島田昌典・back number',
  'lyrics_url': 'htt

In [24]:
df = pd.DataFrame()
i = 0
for li in data:
    for d in li[:50]:
        for k,v in d.items():
            df.loc[i,k] = v
        i += 1

In [29]:
df.release = pd.to_datetime(df.release)

In [31]:
df.to_pickle(DATA/"all_lyrics.pkl")

In [32]:
df

,title,artist,lyricist,composer,arranger,lyrics_url,lyric,release
0,クリスマスソング,back number,清水依与吏,清水依与吏,back number・小林武史・四家卯大,https://www.uta-net.com/song/195484/,どこかで鐘が鳴って らしくない言葉が浮かんで 寒さが心地よくて あれ なんで恋なんかしてんだ...,2015-11-18
1,花束,back number,清水依与吏,清水依与吏,島田昌典・back number,https://www.uta-net.com/song/113824/,どう思う？これから2人でやっていけると思う？ んんどうかなぁでもとりあえずは 一緒にいたいと...,2011-06-22
2,ハッピーエンド,back number,清水依与吏,清水依与吏,小林武史・back number,https://www.uta-net.com/song/216847/,さよならが喉の奥につっかえてしまって 咳をするみたいにありがとうって言ったの 次の言葉はどこ...,2016-11-16
3,高嶺の花子さん,back number,清水依与吏,清水依与吏,back number・蔦谷好位置,https://www.uta-net.com/song/147577/,君から見た僕はきっと ただの友達の友達 たかが知人bにむけられた 笑顔があれならもう 恐ろし...,2013-06-26
4,ヒロイン,back number,清水依与吏,清水依与吏,小林武史・back number,https://www.uta-net.com/song/177145/,君の毎日に 僕は似合わないかな 白い空から 雪が落ちた 別にいいさと 吐き出したため息が 少...,2015-01-21
...,...,...,...,...,...,...,...,...
2445,STEP you,浜崎あゆみ,ayumi hamasaki,Kazuhiro Hara,CMJK,https://www.uta-net.com/song/25903/,どうって事ない会話のやり取り ふとした瞬間に見せる仕草 忘れないようにって思わなくても あた...,2005-04-20
2446,ourselves,浜崎あゆみ,ayumi hamasaki,BOUNCEBACK,CMJK,https://www.uta-net.com/song/17497/,ぎゅっとしてみたり じんとしたり しゅんとしたり またぎゅっとしてみたり しゃんとしたり つ...,2003-07-09
2447,NEVER EVER,浜崎あゆみ,ayumi hamasaki,CREA,,https://www.uta-net.com/song/12815/,いつか生まれる前からきっと 変わらないモノを探し続けては 見つけて 失って 時に 人をキズつ...,2001-03-07
2448,Together When...,浜崎あゆみ,ayumi hamasaki,Kunio Tago,,https://www.uta-net.com/song/58845/,僕達は心に同じ 傷跡を残しながら 背を向けたまま振り返らずに そっと強く歩き出しました 変わ...,2007-12-05


In [ ]:

# 概要出力
topic_info = topic_model.get_topic_info()
topic_info.to_csv(OUTPUT_DIR / "topic_info.csv", index=False, encoding="utf-8-sig")

# 各トピック上位語
# topic_info['Topic'] から -1 は "Other"（雑多）なので除外して見るのがコツ
for topic_id in topic_info["Topic"].tolist():
    if topic_id == -1:
        continue
    words = topic_model.get_topic(topic_id)
    # words は (単語, 重み) のリスト
    pd.DataFrame(words, columns=["word", "weight"]).to_csv(
        OUTPUT_DIR / f"topic_{topic_id}_words.csv", index=False, encoding="utf-8-sig"
    )

# 文書ごとのトピック割当
out_df = pd.DataFrame({
    "text": docs,
    "topic": topics
})
out_df.to_csv(OUTPUT_DIR / "docs_topics.csv", index=False, encoding="utf-8-sig")

# 視覚化（HTML）: UMAP図/バー図/階層図など
try:
    fig = topic_model.visualize_topics()
    fig.write_html(str(OUTPUT_DIR / "viz_topics.html"))

    fig = topic_model.visualize_barchart(top_n_topics=15)
    fig.write_html(str(OUTPUT_DIR / "viz_barchart.html"))

    fig = topic_model.visualize_hierarchy()
    fig.write_html(str(OUTPUT_DIR / "viz_hierarchy.html"))
except Exception as e:
    print("Visualization skipped:", e)

print("Done. outputs/ を確認してください。")